In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import time
from scipy.optimize import minimize, differential_evolution
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_table('occupancy_data/datatest.txt', sep=',')

In [3]:
df2 = pd.read_table('occupancy_data/datatest2.txt', sep=',')

In [4]:
df_ = pd.read_table('occupancy_data/datatraining.txt', sep=',')

In [5]:
df_.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
2,2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,0.004783,1
3,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
4,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
5,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [6]:
df2.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-11 14:48:00,21.7600,31.133333,437.333333,1029.666667,0.005021,1
2,2015-02-11 14:49:00,21.7900,31.000000,437.333333,1000.000000,0.005009,1
3,2015-02-11 14:50:00,21.7675,31.122500,434.000000,1003.750000,0.005022,1
4,2015-02-11 14:51:00,21.7675,31.122500,439.000000,1009.500000,0.005022,1
5,2015-02-11 14:51:59,21.7900,31.133333,437.333333,1005.666667,0.005030,1


In [7]:
df.shape, df2.shape, df_.shape

((2665, 7), (9752, 7), (8143, 7))

In [8]:
data = pd.concat([df, df2, df_], axis=0)
data.shape

(20560, 7)

In [9]:
data.drop_duplicates(inplace=True)

In [10]:
data.shape

(20560, 7)

In [11]:
df = data

In [12]:
df.shape

(20560, 7)

In [13]:
df['Occupancy'].value_counts()

0    15810
1     4750
Name: Occupancy, dtype: int64

In [14]:
df.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
140,2015-02-02 14:19:00,23.7000,26.272,585.200000,749.200000,0.004764,1
141,2015-02-02 14:19:59,23.7180,26.290,578.400000,760.400000,0.004773,1
142,2015-02-02 14:21:00,23.7300,26.230,572.666667,769.666667,0.004765,1
143,2015-02-02 14:22:00,23.7225,26.125,493.750000,774.750000,0.004744,1
144,2015-02-02 14:23:00,23.7540,26.200,488.600000,779.000000,0.004767,1


In [15]:
X = df.drop(columns=['Occupancy', 'date']).values
y = df['Occupancy'].values

In [16]:
for i in range(X.shape[1]):
    X[:,i]=(X[:,i]-X[:,i].min())/(X[:,i].max()-X[:,i].min())

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [18]:
X_train.shape

(14392, 5)

In [19]:
y_train.sum() / len(y_train)

0.23103112840466927

In [20]:
from Seniority_committee import Seniority_committee
sc = Seniority_committee(N=500)
sc.fit(X_train, y_train, optim_method='TNC', verbose=2)

Making hyperplane number 1
X_train.shape[0] = 14392
k = 10
L = 1024
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -3.6303130399761976
Time taken for optimization: 4.715996265411377
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -9437.110231715686
Time taken for optimization: 18.183996200561523
X_1.shape[0] = 424
X_0.shape[0] = 9687

Making hyperplane number 2
X_train.shape[0] = 4705
k = 10
L = 1024
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -15.037350491795223
Time taken for optimization: 2.143998861312866
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -5.236483948670424
Time taken for optimization: 3.6240005493164062
X_1.shape[0] = 496
X_0.shape[0] = 259
Cutted data shape is no

First approximation is obtained
The minimum of the loss function: -121.05021718673439
Time taken for optimization: 1.8539679050445557
X_1.shape[0] = 116
X_0.shape[0] = 133
Cutted data shape is not enough

k = 6
L = 64
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.19338328661311904
Time taken for optimization: 0.7890322208404541
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -116.55608968724789
Time taken for optimization: 1.9480006694793701
X_1.shape[0] = 172
X_0.shape[0] = 133
Cutted data shape is not enough

k = 5
L = 32
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.2667439610076119
Time taken for optimization: 0.5660033226013184
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss functio

In [21]:
train_proba = sc.predict_proba(X_train)
test_proba = sc.predict_proba(X_test)
train_preds = sc.predict(X_train)
test_preds = sc.predict(X_test)
print('Train Gini = {}'.format(2 * roc_auc_score(y_train, train_proba) - 1))
print('Test Gini = {}'.format(2 * roc_auc_score(y_test, test_proba) - 1))
print('Train F1 = {}'.format(f1_score(y_train, train_preds)))
print('Test F1 = {}'.format(f1_score(y_test, test_preds)))
print('Train accuracy = {}'.format(accuracy_score(y_train, train_preds)))
print('Test accuracy = {}'.format(accuracy_score(y_test, test_preds)))

Time taken to predict the targets: 12.790998935699463
Time taken to predict the targets: 4.693029165267944
Time taken to predict the targets: 10.15697169303894
Time taken to predict the targets: 6.107002019882202
Train Gini = 0.9927775252715687
Test Gini = 0.9925055945788992
Train F1 = 0.9771784232365145
Test F1 = 0.9725026105116602
Train accuracy = 0.9892996108949417
Test accuracy = 0.9871919584954605


In [22]:
tree = DecisionTreeClassifier(max_depth=4)
tree.fit(X_train, y_train)
train_proba = tree.predict_proba(X_train)
test_proba = tree.predict_proba(X_test)
train_preds = tree.predict(X_train)
test_preds = tree.predict(X_test)
print('Train Gini = {}'.format(2 * roc_auc_score(y_train, train_proba[:, 1]) - 1))
print('Test Gini = {}'.format(2 * roc_auc_score(y_test, test_proba[:, 1]) - 1))
print('Train F1 = {}'.format(f1_score(y_train, train_preds)))
print('Test F1 = {}'.format(f1_score(y_test, test_preds)))
print('Train Accuracy = {}'.format(accuracy_score(y_train, train_preds)))
print('Test Accuracy = {}'.format(accuracy_score(y_test, test_preds)))

Train Gini = 0.9940520045029897
Test Gini = 0.9892012975724152
Train F1 = 0.979459139943845
Test F1 = 0.9764705882352942
Train Accuracy = 0.9903418565869928
Test Accuracy = 0.9889753566796369


In [23]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
train_proba = logreg.predict_proba(X_train)
test_proba = logreg.predict_proba(X_test)
train_preds = logreg.predict(X_train)
test_preds = logreg.predict(X_test)
print('Train Gini = {}'.format(2 * roc_auc_score(y_train, train_proba[:, 1]) - 1))
print('Test Gini = {}'.format(2 * roc_auc_score(y_test, test_proba[:, 1]) - 1))
print('Train F1 = {}'.format(f1_score(y_train, train_preds)))
print('Test F1 = {}'.format(f1_score(y_test, test_preds)))
print('Train Accuracy = {}'.format(accuracy_score(y_train, train_preds)))
print('Test Accuracy = {}'.format(accuracy_score(y_test, test_preds)))

Train Gini = 0.9891051021427246
Test Gini = 0.9910066543123568
Train F1 = 0.9771756736857605
Test F1 = 0.9765355417529331
Train Accuracy = 0.9892301278488049
Test Accuracy = 0.9889753566796369


In [24]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
train_proba = rf.predict_proba(X_train)
test_proba = rf.predict_proba(X_test)
train_preds = rf.predict(X_train)
test_preds = rf.predict(X_test)
print('Train Gini = {}'.format(2 * roc_auc_score(y_train, train_proba[:, 1]) - 1))
print('Test Gini = {}'.format(2 * roc_auc_score(y_test, test_proba[:, 1]) - 1))
print('Train F1 = {}'.format(f1_score(y_train, train_preds)))
print('Test F1 = {}'.format(f1_score(y_test, test_preds)))
print('Train Accuracy = {}'.format(accuracy_score(y_train, train_preds)))
print('Test Accuracy = {}'.format(accuracy_score(y_test, test_preds)))

Train Gini = 1.0
Test Gini = 0.9969211876412518
Train F1 = 1.0
Test F1 = 0.98353765323993
Train Accuracy = 1.0
Test Accuracy = 0.9923800259403373


In [25]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
train_proba = lgbm.predict_proba(X_train)
test_proba = lgbm.predict_proba(X_test)
train_preds = lgbm.predict(X_train)
test_preds = lgbm.predict(X_test)
print('Train Gini = {}'.format(2 * roc_auc_score(y_train, train_proba[:, 1]) - 1))
print('Test Gini = {}'.format(2 * roc_auc_score(y_test, test_proba[:, 1]) - 1))
print('Train F1 = {}'.format(f1_score(y_train, train_preds)))
print('Test F1 = {}'.format(f1_score(y_test, test_preds)))
print('Train Accuracy = {}'.format(accuracy_score(y_train, train_preds)))
print('Test Accuracy = {}'.format(accuracy_score(y_test, test_preds)))

Train Gini = 0.9999982335888513
Test Gini = 0.9986260823891902
Train F1 = 0.998798076923077
Test F1 = 0.982098982098982
Train Accuracy = 0.9994441356309061
Test Accuracy = 0.9917315175097277
